In [4]:
import requests
import pandas as pd

### Locations per cluster
The first step is to slightly modify the loading of the clusters

In [8]:
def load_api_data(prnt=False):
    """
    This function loads in information on the current composition of container
    clusters in Amsterdam. It uses the API from data.amsterdam.nl (available at
    'https://api.data.amsterdam.nl/vsd/afvalclusters'). It returns the coordinates,
    amount and volume of different fractions and the address of the clusters. As
    a check, it is determined whether or not the cluster is currently active.
    Returns:
    - df containing coordinates, dict-like amount and volume per fraction and
    address.
    """
    x_coordinates = []
    y_coordinates = []
    aantal = []
    volumes = []
    adresses = []
    buurt = []

    link = 'https://api.data.amsterdam.nl/vsd/afvalclusters'

    while link != None: #This is the case on the last page of the API
        if prnt: # Can be used for some kind of monitoring of progres
            print(link)
        response = requests.get(link)
        output = response.json()
        for result in output['results']:
            if result['cluster_datum_einde_cluster'] == None: #Als het cluster nog actief is
                x_coordinates.append(str(result['cluster_geometrie']['coordinates'][0]))
                y_coordinates.append(str(result['cluster_geometrie']['coordinates'][1]))
                aantal.append(result['cluster_fractie_aantal'])
                volumes.append(result['cluster_fractie_volume'])
                adresses.append(result['bag_adres_openbare_ruimte_naam'])
                buurt.append(result['gbd_buurt_code'])
        try:
            link = output['_links']['next']['href'] #Retrieve link for next page
        except:
            link = None #True for last page of API

    df_clusters = pd.DataFrame([x_coordinates, y_coordinates, aantal, volumes, adresses, buurt]).T
    df_clusters = df_clusters.rename(columns={0: 'cluster_x', 1:'cluster_y', 2:'aantal_per_fractie', 3:'volume_per_fractie', 4: 'street_name', 5:'buurt'})
    # Transform coordinates of clusters to ints, as this helps easing join
    df_clusters['cluster_x'] = df_clusters['cluster_x'].astype('float').round(0).astype('int')
    df_clusters['cluster_y'] = df_clusters['cluster_y'].astype('float').round(0).astype('int')
    df_clusters['wijk'] = df_clusters['buurt'].str[:3]
    df_clusters['stadsdeel'] = df_clusters['buurt'].str[0]
    return df_clusters

In [9]:
df_clusters = load_api_data_neigborhood()

In [20]:
df_clusters

,cluster_x,cluster_y,aantal_per_fractie,volume_per_fractie,street_name,buurt,wijk,stadsdeel
0,112740,487843,"[Glas:1, Papier:1, Plastic:1]","[Glas:3 m3, Papier:3 m3, Plastic:4 m3]",Raasdorperweg,F80b,F80,F
1,112973,485077,[Rest:2],[Rest:8 m3],Schotse Hooglanden,F84b,F84,F
2,113012,484958,"[Rest:1, Papier:1]","[Rest:5 m3, Papier:3 m3]",Pyreneeën,F84b,F84,F
3,113044,485173,[Rest:2],[Rest:6 m3],Ben Wyvis,F84b,F84,F
4,113049,487544,"[Glas:1, Papier:1, Plastic:1]","[Glas:3 m3, Papier:3 m3, Plastic:3 m3]",Osdorperweg,F80b,F80,F
...,...,...,...,...,...,...,...,...
5691,129621,479867,[Glas:1],[Glas:3 m3],Winterstraat,T98a,T98,T
5692,129755,479991,"[Glas:1, Papier:1, Plastic:1, Textiel:1]","[Glas:3 m3, Papier:4 m3, Plastic:4 m3, Textiel...",Winterstraat,T98a,T98,T
5693,129798,479676,"[Glas:1, Papier:1, Plastic:1]","[Glas:4 m3, Papier:4 m3, Plastic:4 m3]",Zandpad-Driemond,T98a,T98,T
5694,129898,479851,[Rest:1],[Rest:5 m3],Lentestraat,T98a,T98,T
